In [2]:
import pandas as pd
import numpy as np
from datetime import datetime


In [3]:
all_seasons = pd.read_csv('all_seasons.csv')
all_seasons = all_seasons[['datetime','conditions']]
all_seasons.head()


,datetime,conditions
0,1/1/2000,Partially cloudy
1,2/1/2000,Clear
2,3/1/2000,Clear
3,4/1/2000,Clear
4,5/1/2000,Clear


In [4]:
simplifier = {'Overcast':'cloudy','Partially cloudy':'cloudy','Clear':'sunny','Rain':'rainy','Rain,Partially cloudy':'rainy','Rain,Overcast':'rainy'}
all_seasons['condition']= all_seasons['conditions'].map(simplifier)
all_seasons.head()

,datetime,conditions,condition
0,1/1/2000,Partially cloudy,cloudy
1,2/1/2000,Clear,sunny
2,3/1/2000,Clear,sunny
3,4/1/2000,Clear,sunny
4,5/1/2000,Clear,sunny


In [5]:
all_seasons = all_seasons[['datetime','condition']]
all_seasons.head()

,datetime,condition
0,1/1/2000,cloudy
1,2/1/2000,sunny
2,3/1/2000,sunny
3,4/1/2000,sunny
4,5/1/2000,sunny


In [6]:
train_start_date = '1/1/2000'
train_end_date = '1/1/2022'
train_start_date = pd.to_datetime(train_start_date, format='%d/%m/%Y')
train_end_date = pd.to_datetime(train_end_date, format='%d/%m/%Y')
all_seasons['datetime'] = pd.to_datetime(all_seasons['datetime'], format='%d/%m/%Y')
all_seasons_train = all_seasons.loc[(all_seasons['datetime'] >= train_start_date) & (all_seasons['datetime'] <= train_end_date)]
all_seasons_train = all_seasons_train.sort_values(by='datetime')
all_seasons_train = all_seasons_train.reset_index(drop=True)
all_seasons_train


,datetime,condition
0,2000-01-01,cloudy
1,2000-01-02,sunny
2,2000-01-03,sunny
3,2000-01-04,sunny
4,2000-01-05,sunny
...,...,...
8032,2021-12-28,NaN
8033,2021-12-29,NaN
8034,2021-12-30,NaN
8035,2021-12-31,cloudy


In [7]:
#set conditions count S=Sunny R=Rain C=cloudy 
S_after_S_count=0.0
S_after_R_count=0.0
S_after_C_count=0.0

C_after_S_count=0.0
C_after_R_count=0.0
C_after_C_count=0.0

R_after_S_count=0.0
R_after_R_count=0.0
R_after_C_count=0.0

In [8]:
#count
all_seasons_train['condition_shift'] = all_seasons_train['condition'].shift(-1)

for i in range (len(all_seasons_train)):
    if all_seasons_train.loc[i, 'condition'] == 'sunny' and all_seasons_train.loc[i,'condition_shift'] == 'sunny': 
        S_after_S_count += 1
    elif all_seasons_train.loc[i, 'condition'] == 'sunny' and all_seasons_train.loc[i,'condition_shift'] == 'rainy': 
        S_after_R_count += 1
    elif all_seasons_train.loc[i, 'condition'] == 'sunny' and all_seasons_train.loc[i,'condition_shift'] == 'cloudy': 
        S_after_C_count += 1
    elif all_seasons_train.loc[i, 'condition'] == 'rainy' and all_seasons_train.loc[i,'condition_shift'] == 'rainy': 
        R_after_R_count += 1
    elif all_seasons_train.loc[i, 'condition'] == 'rainy' and all_seasons_train.loc[i,'condition_shift'] == 'sunny': 
        R_after_S_count += 1 
    elif all_seasons_train.loc[i, 'condition'] == 'rainy' and all_seasons_train.loc[i,'condition_shift'] == 'cloudy': 
        R_after_C_count += 1
    elif all_seasons_train.loc[i, 'condition'] == 'cloudy' and all_seasons_train.loc[i,'condition_shift'] == 'rainy': 
        C_after_R_count += 1
    elif all_seasons_train.loc[i, 'condition'] == 'cloudy' and all_seasons_train.loc[i,'condition_shift'] == 'sunny': 
        C_after_S_count += 1
    elif all_seasons_train.loc[i, 'condition'] == 'cloudy' and all_seasons_train.loc[i,'condition_shift'] == 'cloudy': 
        C_after_C_count += 1

In [9]:
current_S_total = S_after_S_count + R_after_S_count + C_after_S_count
current_R_total = S_after_R_count + R_after_R_count + C_after_R_count
current_C_total = S_after_C_count + R_after_C_count + C_after_C_count
print(current_C_total)


3270.0


In [10]:
S_after_S_prob = S_after_S_count / current_S_total
S_after_R_prob = S_after_R_count / current_R_total 
S_after_C_prob = S_after_C_count / current_C_total

R_after_R_prob = R_after_R_count/current_R_total
R_after_S_prob = R_after_S_count/current_S_total
R_after_C_prob = R_after_C_count/current_C_total

C_after_C_prob = C_after_C_count/current_C_total
C_after_S_prob = C_after_S_count/current_S_total
C_after_R_prob = C_after_R_count/current_R_total



In [11]:
print(S_after_S_count)
print(S_after_C_count)
print(S_after_R_count)

2328.0
830.0
56.0


In [12]:
print(S_after_S_prob)
print(S_after_R_prob)
print(S_after_C_prob)
print(R_after_S_prob)
print(R_after_R_prob)
print(R_after_C_prob)
print(C_after_S_prob)
print(C_after_R_prob)
print(C_after_C_prob)
print(S_after_S_prob+R_after_S_prob+C_after_S_prob)
print(S_after_R_prob+R_after_R_prob+C_after_R_prob)
print(S_after_C_prob+R_after_C_prob+C_after_C_prob)

0.7444835305404541
0.5
0.25382262996941896
0.02910137511992325
0.1875
0.007339449541284404
0.22641509433962265
0.3125
0.7388379204892966
1.0
1.0
1.0


In [13]:
state = {'S':'sunny' , 'R':'rainy' ,'C':'cloudy'}
transition_name = [['SS','RS','CS'],['SR','RR','CR'],['SC','RC','CC']]
transition_matrix = [[S_after_S_prob,R_after_S_prob,C_after_S_prob],
                     [S_after_R_prob,R_after_R_prob,C_after_R_prob],
                     [S_after_C_prob,R_after_C_prob,C_after_C_prob]]
t_array = np.array(transition_matrix)

In [14]:
def predict_weather(days):
    predicted_weather = []
    state_to_index = {'S':0,'R':1,'C':2}
    current_state ='C'

    for _ in range(days):
        predicted_weather.append(state[current_state])
        transition_probs = t_array[state_to_index[current_state]]
        next_state_index = np.random.choice([0,1,2],p=transition_probs)
        current_state = list(state.keys())[list(state_to_index.values()).index(next_state_index)]
    return predicted_weather

In [15]:
predicted_result = predict_weather(8341)
print(predicted_result)

['cloudy', 'sunny', 'sunny', 'sunny', 'sunny', 'cloudy', 'cloudy', 'sunny', 'sunny', 'sunny', 'sunny', 'cloudy', 'cloudy', 'cloudy', 'cloudy', 'cloudy', 'sunny', 'sunny', 'sunny', 'sunny', 'cloudy', 'cloudy', 'cloudy', 'cloudy', 'cloudy', 'sunny', 'sunny', 'cloudy', 'cloudy', 'sunny', 'sunny', 'sunny', 'sunny', 'sunny', 'sunny', 'sunny', 'cloudy', 'cloudy', 'cloudy', 'cloudy', 'sunny', 'cloudy', 'sunny', 'sunny', 'sunny', 'sunny', 'cloudy', 'cloudy', 'cloudy', 'cloudy', 'sunny', 'cloudy', 'cloudy', 'cloudy', 'sunny', 'sunny', 'sunny', 'sunny', 'cloudy', 'sunny', 'rainy', 'sunny', 'sunny', 'cloudy', 'cloudy', 'cloudy', 'cloudy', 'sunny', 'sunny', 'sunny', 'sunny', 'sunny', 'sunny', 'sunny', 'cloudy', 'cloudy', 'sunny', 'sunny', 'sunny', 'sunny', 'sunny', 'sunny', 'sunny', 'sunny', 'cloudy', 'cloudy', 'cloudy', 'cloudy', 'cloudy', 'cloudy', 'cloudy', 'cloudy', 'cloudy', 'sunny', 'sunny', 'sunny', 'sunny', 'sunny', 'sunny', 'sunny', 'sunny', 'sunny', 'sunny', 'cloudy', 'cloudy', 'sunny', 

In [16]:
def find_accuracy(predicted_result):
    correct_count = 0.0
    for i in range(len(predicted_result)):
        if predicted_result.loc[i,'condition'] == predicted_result.loc[i,'predicted_result']:
            correct_count += 1
    correct_prop = correct_count/len(predicted_result)
    return correct_prop

In [17]:
all_seasons['predicted_result'] = predicted_result
all_seasons = all_seasons[['datetime','condition','predicted_result']]
print(all_seasons)
print('accuracy:',find_accuracy(all_seasons))

       datetime condition predicted_result
0    2000-01-01    cloudy           cloudy
1    2000-01-02     sunny            sunny
2    2000-01-03     sunny            sunny
3    2000-01-04     sunny            sunny
4    2000-01-05     sunny            sunny
...         ...       ...              ...
8336 2022-10-28     sunny           cloudy
8337 2022-10-29     sunny           cloudy
8338 2022-10-30     sunny           cloudy
8339 2022-10-31     sunny            sunny
8340 2022-11-01    cloudy           cloudy

[8341 rows x 3 columns]
accuracy: 0.4251288814290852
